# O R² score não varia entre 0 e 1

O coeficiente de determinação, conhecido como $R^2$, é uma métrica fundamental em análises de regressão. Contudo, sua definição e interpretação nem sempre são diretas. De fato, existem várias maneiras de definir o $R^2$ e, embora todas sejam equivalentes, cada uma apresenta uma nuance interpretativa diferente. Algumas dessas interpretações são mais intuitivas, facilitando uma compreensão imediata dos valores possíveis, enquanto outras podem levar a equívocos.

A versão atual do scikit-learn, em sua docstring do [`sklearn.metrics.r2_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html), menciona que o $R^2$ pode variar de menos infinito até 1. Contudo, não é raro encontrar cientistas de dados que afirmam que a amplitude de valores possíveis para o $R^2$ está estritamente entre 0 e 1. Uma das razões para essa discrepância está na interpretação clássica do $R^2$, que é tradicionalmente entendida como a proporção da variância explicada pelo modelo em relação à variância total da variável alvo [[1](#bibliography)].

Ao longo deste texto, abordarei a interpretação que considero mais esclarecedora e relevante. Com ela, espero elucidar algumas peculiaridades do $R^2$ e destacar sua importância como uma métrica robusta, frequentemente consultada em problemas de regressão.

___

## Erro quadrático médio e a escolha de um modelo constante

O $R^2$ é uma métrica comum em regressão. No entanto, frequentemente a primeira métrica introduzida para problemas de regressão é o Erro Quadrático Médio (MSE, do inglês Mean Squared Error). O MSE de um modelo $h$ em um conjunto de dados $S = \{ (x_i, y_i) \}_{i=1}^n$ é definido por

$$ \textrm{MSE}(h) = \frac{1}{n} \sum_{i=1}^n \left(y_i - h(x_i)\right)^2, $$

onde optamos por não denotar a dependência de $S$ a fim de manter a notação mais simplificada.

Diante dessa definição, surge uma pergunta intrigante: se você tivesse que criar um modelo que fosse meramente uma constante, qual valor você escolheria? Muitos poderiam responder que escolheriam a média, que é, de fato, uma das respostas corretas. No entanto, por que não considerar a mediana, a moda ou alguma outra estatística descritiva?

A resposta a essa questão está intrinsecamente ligada à função de custo que desejamos otimizar. Essa escolha é, de fato, um problema da teoria de decisão [[2](#bibliography)]. Por exemplo, se o objetivo é otimizar o MSE, então precisaríamos escolher um $\alpha \in \mathbb{R}$ tal que $h_\alpha(x) = \alpha$ minimize o $\textrm{MSE}(h_\alpha)$. Matematicamente, isso é expresso como

$$ \alpha^* = \arg\min_{\alpha \in \mathbb{R}} \textrm{MSE}(h_\alpha) = \arg\min_{\alpha \in \mathbb{R}} \left( \frac{1}{n} \sum_{i=1}^n \left(y_i - \alpha\right)^2 \right). $$

Essa função pode parecer complexa à primeira vista, mas se torna mais simples ao considerar apenas $\alpha$ como variável livre, que é como abordamos esse problema de otimização. Ao expandir o quadrado e realizar o somatório, temos uma função polinomial de grau 2 em $\alpha$ da forma

$$\frac{1}{n} \sum_{i=1}^n \left(y_i - \alpha\right)^2 = \frac{1}{n} \sum_{i=1}^n \left(y_i^2 -2\alpha y_i + \alpha^2 \right) = \alpha^2  + \left(\frac{-2}{n} \sum_{i=1}^n y_i\right) \alpha+ \left(\frac{1}{n} \sum_{i=1}^n y_i^2\right).$$

Em uma função quadrática da forma $(a\,\alpha^2 + b\,\alpha + c)$, onde $a>0$, o mínimo ocorre no vértice da parábola, localizado em $\frac{-b}{2a}$. Assim, no nosso contexto, o mínimo é

$$ \alpha^* = \frac{\left(\frac{-2}{n} \sum_{i=1}^n y_i\right)}{-2} = \frac{1}{n} \sum_{i=1}^n y_i = \bar{y}.$$

Isso significa que, ao minimizar o MSE, o valor constante ótimo é a média do target $\bar{y}$ para esse conjunto. Encorajo a validação desse resultado a partir da utilização de outras técnicas de otimização irrestrita como: identificação de pontos críticos sguida da análise da concavidade da função.

Este comportamento muda ao considerarmos outras métricas [[3](#bibliography)]. Por exemplo, para minimizar o Mean Absolute Error (MAE), o valor constante que o minimiza é a mediana, enquanto o valor que minimiza a acurácia é a moda, e para a pinball loss é o quantil associado. Importante ressaltar que, se considerarmos `sample_weight`, todas essas estatísticas devem ser calculadas de forma ponderada.

$\oint$ _Isso é usado na definição de valores para os nós das árvores de decisão. Observando o código do scikit-learn para árvores, notamos que, dependendo do critério, o [`node_value`](https://github.com/scikit-learn/scikit-learn/blob/d7a114413d1f11bf5f7029cd519c9a29a66b1890/sklearn/tree/_criterion.pyx#L1036) pode variar. Ele é ajustado para refletir o valor que minimiza a perda quando o nó faz uma previsão constante. Por exemplo, para o critério MSE a previsão da folha é a média do target dos exemplos de treinamento que caem nessa folha, enquanto para o critério MAE é a mediana._

$\oint$ _Na prática, um modelo que prevê a média do target não é viável porque para calcular a média do conjunto de teste você precisaria conhecer os valores de $y_i$ dessa amostra. No entanto, essa perspectiva é útil para comparar um modelo básico com o seu modelo, como discutiremos a seguir._

___

## R² como comparação entre seu modelo e um modelo simples

Suponha que eu desenvolva um modelo para prever a idade de uma pessoa com base em seu comportamento online e obtenha um MSE de 25 anos ao quadrado. Esse número isoladamente pode não ser muito informativo. Uma maneira de interpretá-lo é calcular o Root Mean Squared Error, ou seja, $\textrm{RMSE} = \sqrt{\textrm{MSE}}$, resultando em um erro de aproximadamente 5 anos. Esse valor é mais intuitivo (confesso que, internamente, costumo pensar em termos de MAE), mas ainda não fornece uma comparação relativa como "será que é possível obter um valor significativamente menor do que este?". O $R^2$ pode não responder essa pergunta diretamente, mas ajuda nessa avaliação.

Já discutimos um modelo simples que pode servir como referência. Imagine que o modelo baseado na média já produza um MSE de 30 anos ao quadrado. Subitamente, nosso modelo anterior, que poderia parecer excelente, não se destaca tanto. Se um modelo simples já alcança um MSE apenas um pouco maior que o modelo atual, vale a pena implementar o modelo mais complexo em um ambiente de produção?

A interpretação que faço do $R^2$ é justamente essa comparação. Sua fórmula é

$$ R^2(h) = 1 - \frac{\textrm{MSE}(h)}{\textrm{MSE}(\bar{y})},$$

onde $\bar{y}$ representa a média do target no conjunto $S$ em que estamos avaliando o modelo.

Com isso, podemos entender os valores possíveis de $R^2$:

- Se $R^2 = 1$, significa que $\textrm{MSE}(h) = 0$; ou seja, o modelo é perfeito.
- Se $R^2 = 0$, temos $\textrm{MSE}(h) = \textrm{MSE}(\bar{y})$, indicando que nosso modelo é tão eficaz quanto um modelo que simplesmente fornece a média do target.
- Para um $R^2$ entre 0 e 1, temos $0 < \textrm{MSE}(h) < \textrm{MSE}(\bar{y})$. Isso indica que o modelo tem um erro maior que zero, mas menor que o de um modelo constante baseado na média.
- Um $R^2$ negativo sugere que $\textrm{MSE}(h) > \textrm{MSE}(\bar{y})$, ou seja, nosso modelo é menos preciso do que um que sempre fornece a média.

Essa interpretação auxilia na compreensão dos valores obtidos ao usar a função [`sklearn.metrics.r2_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html). No exemplo anterior, teríamos um $R^2$ de $(1 - 25/30) \approx 0.17$, indicando um modelo que supera o modelo simples, mas não de forma muito significativa.

In [1]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    *fetch_california_housing(return_X_y=True),
    test_size=0.33,
    random_state=42,
)

lr = LinearRegression().fit(X_train, y_train)

def evaluate_model(y_true, y_pred):
    print(f"MSE: {mean_squared_error(y_true, y_pred)}")
    print(f"R^2: {r2_score(y_true, y_pred)}")
    
y_pred_lr =  lr.predict(X_test)
evaluate_model(y_test, y_pred_lr)

MSE: 0.5369686543372444
R^2: 0.5970494128783965


In [2]:
y_mean_test = y_test.mean() * np.ones_like(y_test)
evaluate_model(y_test, y_mean_test)

MSE: 1.3325918152222385
R^2: 0.0


In [3]:
y_pred_horrible_model = np.zeros_like(y_test)
evaluate_model(y_test, y_pred_horrible_model)

MSE: 5.6276808369101445
R^2: -3.2231092616846126


Embora um modelo com $R^2$ igual a zero possa parecer o patamar mínimo alcançável, na realidade, essa métrica se utiliza de um modelo baseline com vazamento de dados. Na prática, construímos nossos modelos usando dados de treinamento e, em cenários sujeitos a "dataset shift", pode haver mudanças significativas em estatísticas fundamentais, como a média.

In [4]:
y_mean_train = y_train.mean() * np.ones_like(y_test)
evaluate_model(y_test, y_mean_train)

MSE: 1.3326257277946882
R^2: -2.5448582275933163e-05


Independentemente dessas nuances, interpretar o $R^2$ dessa forma oferece um valioso mindset de comparação. É sempre fundamental comparar seu modelo com baselines simples, seja com regras de negócio estabelecidas ou com modelos mais básicos, como uma constante.

___

## Generalização do R² além do MSE

A noção de comparação com um modelo básico ou simples pode ser facilmente generalizada para outras métricas, desde que saibamos quais estatísticas usar como baseline. Considerando isso, proponho a extensão dessa ideia para o MAE utilizando a mediana $\tilde{y}$ como modelo baseline

$$ R^2_{\textrm{MAE}}(h) = 1 - \frac{\textrm{MAE}(h)}{\textrm{MAE}(\tilde{y})}, $$

onde 

$$ \textrm{MAE}(h) = \frac{1}{n} \sum_{i=1}^n \left| y_i - h(x_i) \right|. $$

Assim, o $R^2_{\textrm{MAE}}$ oferece uma forma de avaliar o desempenho do modelo em relação a um baseline simples, usando o MAE como métrica de erro.

In [5]:
from sklearn.metrics import mean_absolute_error

def r2_score_mae(y_true, y_pred, *args, **kwargs):
    mae_model = mean_absolute_error(y_true=y_true, y_pred=y_pred, *args, **kwargs)
    y_median_true = np.median(y_true) * np.ones_like(y_true)
    mae_median = mean_absolute_error(
        y_true=y_true, y_pred=y_median_true, *args, **kwargs
    )
    return 1 - mae_model / mae_median

def evaluate_model_mae(y_true, y_pred):
    print(f"MAE: {mean_absolute_error(y_true, y_pred)}")
    print(f"R^2_MAE: {r2_score_mae(y_true, y_pred)}")

evaluate_model_mae(y_test, y_pred_lr)

MAE: 0.5295710106684688
R^2_MAE: 0.40256278728026484


In [6]:
y_median_test = np.median(y_test) * np.ones_like(y_test)
evaluate_model_mae(y_test, y_median_test)

MAE: 0.8864044612448619
R^2_MAE: 0.0


___

## Considerações finais

O equívoco de que o $R^2$ varia somente entre 0 e 1 origina-se de uma interpretação simplificada do seu significado mais comum: a proporção da variância do alvo que é explicada pelas variáveis independentes, o que sugere que o valor esteja entre 0% e 100%. Na prática, em muitos casos, o $R^2$ realmente se encontra nesse intervalo. Contudo, em situações nas quais o modelo é inferior a um simples modelo horizontal (isto é, uma linha reta que representa a média), o $R^2$ pode ter valores negativos. Esse cenário negativo é frequentemente subestimado pela comunidade estatística, pois geralmente está associado a situações de overfitting. Raramente, uma regressão linear que tende a sofrer underfitting será inferior ao modelo horizontal que está incluído no espaço de hipóteses da regressão linear.

Durante esse post, analisamos alguns dos motivos pelos quais o $R^2$ é uma métrica tão interessante e amplamente utilizada em problemas de regressão. Ao compreender a comparação implícita com um modelo baseline, obtemos uma perspectiva valiosa sobre o desempenho relativo do nosso modelo normalizando os valores menos informativos do MSE quando vistos isoladamente. Além disso, a interpretação proposta aqui realmente nos permite entender os valores resultantes de maneira clara e objetiva.

## <a name="bibliography">Referências</a>

[1] [Coefficient of determination. Wikipedia.](https://en.wikipedia.org/wiki/Coefficient_of_determination)

[2] [Introdução à Teoria da Decisão. Fundamentos de Inferência Bayesiana. Victor Fossaluza e Luís Gustavo Esteves.](https://vfossaluza.github.io/InfBayes/TeoDec.html)

[3] [Estimação Pontual. Fundamentos de Inferência Bayesiana. Victor Fossaluza e Luís Gustavo Esteves.](https://vfossaluza.github.io/InfBayes/Estimacao.html#estima%C3%A7%C3%A3o-pontual)